- notebook/requirements.txt: EDA를 위한 종속성 관리
- autoEDA: llama2로 특수 문자 여부를 판단한 내용

In [7]:
import pandas as pd
import pyarrow as pa
import numpy as np
import string
from collections import Counter
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [8]:
file_path = '../data/train_dataset/train/dataset.arrow'
with pa.memory_map(file_path, 'r') as source:
    table = pa.ipc.open_stream(source).read_all()
df: pd.DataFrame = table.to_pandas()
df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


\* 로컬 Ollama 서버로 llama2 활용하는 방법 (macOS 기준)
1. Ollama 설치
    ```
    brew install ollama
    ```
2. Ollama 서버 실행
    ```
    ollama run llama2
    ```
3. Ollama 서버 중단
    ```
    pkill ollama
    ```

In [9]:
llm = OllamaLLM(model="llama2")

In [10]:
text_analysis_template = """
다음 문서에 따라 주어진 질문에 **오직 한국어로만** 대답해 주세요. 영어로 대답하지 마세요.

### 문서
{context}

### 질문
{question}

오직 정답만 말하고 한국어로만 말해. 정답에서 **가장 중요한 단어만 한국어로** 대답해.
"""

text_analysis_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=text_analysis_template
)

text_analysis_chain = (
    {
        "context": RunnablePassthrough(),
        "question": RunnablePassthrough(),
    }
    | text_analysis_prompt
    | llm
    | StrOutputParser()
)

In [11]:
def normalize_korean_answer(s):
    def remove_punctuation(text):
        exclude = set(string.punctuation + "·、，．？！＂＇〃《》「」『』〔〕“”‘’〈〉【】()[]{}")
        return "".join(ch for ch in text if ch not in exclude)

    def white_space_fix(text):
        return " ".join(text.split())

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punctuation(lower(s)))

def exact_match(prediction, ground_truth):
    return normalize_korean_answer(prediction) == normalize_korean_answer(ground_truth)

# F1 Score calculation
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_korean_answer(prediction).split()
    ground_truth_tokens = normalize_korean_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())

    if num_same == 0:
        return 0.0

    precision = num_same / len(prediction_tokens)
    recall = num_same / len(ground_truth_tokens)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [12]:
exact_matches = []
f1_scores = []

for idx in range(len(df)):
    text = df.iloc[idx]['context']
    question = df.iloc[idx]['question']
    answer = df.iloc[idx]['answers']['text'][0]
    
    result = text_analysis_chain.invoke(
        {
            "context": text,
            "question": question,
            "answer": answer,
        }
    )
    
    predicted_answer = result.strip()
    
    em = exact_match(predicted_answer, answer)
    f1 = f1_score(predicted_answer, answer)

    exact_matches.append(em)
    f1_scores.append(f1)
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"True Answer: {answer}")
    print(f"Exact Match: {em}, F1 score: {f1}")
    print("-" * 40)
    
avg_em = np.mean(exact_matches)
avg_f1 = np.mean(f1_scores)

print(f"Average Exact Match: {avg_em:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")

Question: 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
Predicted Answer: **상원**
True Answer: 하원
Exact Match: False, F1 score: 0.0
----------------------------------------
Question: 현대적 인사조직관리의 시발점이 된 책은?
Predicted Answer: The book that marked the transition from traditional to modern management is "The Practice of Management" by Peter Drucker in 1954.
True Answer: 《경영의 실제》
Exact Match: False, F1 score: 0.0
----------------------------------------
Question: 강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
Predicted Answer: The emperor's letter in 1717 was written for whom?

Answer: The people.
True Answer: 백성
Exact Match: False, F1 score: 0.0
----------------------------------------
Question: 11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
Predicted Answer: 11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?

중국의 영향을 받았다.
True Answer: 중국
Exact Match: False, F1 score: 0.0
----------------------------------------
Question: 명문이 적힌 유물을 구성하는 그릇의 총 개수는?
Predicted Answer: The set of vessels composed of the inscribedware includes 4 pieces in t